In [1]:
!pip install faiss-cpu sentence-transformers PyMuPDF -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.3 MB/s eta 0:00:00


In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import fitz
import os
import numpy as np


In [3]:
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def split_text_into_chunks(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks



In [4]:
pdf_paths = [
    "/content/OUR PAST PART 2 CLASS 8 SOCIAL SCIENCE.pdf",
    "/content/class 9 abc.pdf",
    "/content/History6.pdf",
    "/content/NCERT-Class-10-History.pdf"
]

# Extract and store texts
documents = {}
for path in pdf_paths:
    text = extract_text_from_pdf(path)
    documents[path] = text
    print(f"✅ Extracted {len(text)} characters from {path}")


✅ Extracted 199457 characters from /content/OUR PAST PART 2 CLASS 8 SOCIAL SCIENCE.pdf
✅ Extracted 386175 characters from /content/class 9 abc.pdf
✅ Extracted 208829 characters from /content/History6.pdf
✅ Extracted 428635 characters from /content/NCERT-Class-10-History.pdf


In [5]:
all_chunks = []

for file_path, text in documents.items():
    chunks = split_text_into_chunks(text)
    for i, chunk in enumerate(chunks):
        all_chunks.append({
            "text": chunk,
            "source": file_path,
            "chunk_id": i
        })

print(f"📚 Total chunks created: {len(all_chunks)}")
print("🔍 Sample chunk:\n", all_chunks[0]["text"][:500])


📚 Total chunks created: 3060
🔍 Sample chunk:
 i
SOCIAL SCIENCE
Textbook in History
for Class VIII


2018-19
ii
First Edition
January 2008  Magha 1929
Reprinted
January 2009  Magha 1930
January 2010 Magha 1931
November 2010 Kartika 1932
January 2012 Magha 1933
February 2013 Magha 1934
January 2014 Pausa 1935
December 2014 Pausa 1936
January 2016 Pausa 1937
December 2016 Pausa 1938
January 2018 Magha 1939
PD 650+100T RPS
© National Council of Educational
Research and Training, 2008




55.00
Printed on 80 GSM pa


#Generate Embeddings & Store in FAISS

In [6]:
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
texts = [chunk["text"] for chunk in all_chunks]

embeddings = model.encode(texts, show_progress_bar=True)


Batches:   0%|          | 0/96 [00:00<?, ?it/s]

In [8]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print(f"✅ FAISS index created with {index.ntotal} chunks.")


✅ FAISS index created with 3060 chunks.


In [11]:
metadata = [{"text": chunk["text"], "source": chunk["source"]} for chunk in all_chunks]


#handilng questions and retreves

In [12]:
def search_documents(question, k=5):
    # Convert question into embedding
    question_embedding = model.encode([question])

    # Search top-k closest chunks
    D, I = index.search(np.array(question_embedding), k)

    # Retrieve matching chunks with sources
    results = []
    for i in I[0]:
        result = metadata[i]
        results.append(result)

    return results


In [13]:
question = "What is democracy?"
results = search_documents(question, k=3)

for idx, res in enumerate(results):
    print(f"\n🔹 Result {idx+1} (from: {res['source']})")
    print(res['text'][:700])  # Show first 700 characters



🔹 Result 1 (from: /content/class 9 abc.pdf)
lled power of dynastic rulers. They wanted
to safeguard the rights of individuals against governments. They
argued for a representative, elected parliamentary government, subject
to laws interpreted by a well-trained judiciary that was independent
of rulers and officials. However, they were not ‘democrats’. They
did not believe in universal adult franchise, that is, the right of every
citizen to vote. They felt men of property mainly should have the
vote. They also did not want the vote for wome

🔹 Result 2 (from: /content/History6.pdf)
ago, the people of Athens set up a form of
government, which was called a democracy, which lasted for about 200
years.
All free men over the age of 30 were recognised as full citizens.
There was an assembly that met at least 40 times a year to decide on
important matters.
All citizens could attend these meetings.
Appointments for many positions were made through lottery. All those
who wanted to be chosen gave

 #Generate Human-Like Answers from Retrieved Chunks

In [14]:
def generate_answer(chunks, question):
    answer = "Based on the documents:\n\n"
    for i, chunk in enumerate(chunks):
        answer += f"🔹 Point {i+1}: {chunk['text'][:300].strip()}...\n"
    answer += "\n(Answer based on most relevant matches.)"
    return answer

# Try it
question = "What is democracy?"
results = search_documents(question, k=3)
final_answer = generate_answer(results, question)

print(final_answer)


Based on the documents:

🔹 Point 1: lled power of dynastic rulers. They wanted
to safeguard the rights of individuals against governments. They
argued for a representative, elected parliamentary government, subject
to laws interpreted by a well-trained judiciary that was independent
of rulers and officials. However, they were not ‘dem...
🔹 Point 2: ago, the people of Athens set up a form of
government, which was called a democracy, which lasted for about 200
years.
All free men over the age of 30 were recognised as full citizens.
There was an assembly that met at least 40 times a year to decide on
important matters.
All citizens could attend t...
🔹 Point 3: chosen gave in their names, and then some were
selected through lottery.
Citizens were expected to serve in the army and the navy.
However, women were not considered citizens.
Also, many foreigners, who lived and worked in Athens as merchants
and crafts persons did not have rights as citizens.
Besi...

(Answer based on most relevant

# creating UI

In [15]:
!pip install gradio -q


In [16]:
import gradio as gr

# Main chatbot function
def chatbot_interface(user_query):
    results = search_documents(user_query, k=3)
    answer = generate_answer(results, user_query)
    return answer


In [17]:
with gr.Blocks() as demo:
    gr.Markdown("### 🤖 UPSC Smart Chatbot (NCERT-Powered)")
    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(label="Ask a UPSC Question")
            submit_btn = gr.Button("Get Answer")
        with gr.Column():
            output = gr.Textbox(label="Answer", lines=10)

    submit_btn.click(fn=chatbot_interface, inputs=user_input, outputs=output)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://91d963e5fbeb1eb156.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
